In [83]:
import nltk 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import SGD
import random
import warnings


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sinha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sinha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [86]:
words =[]
classes =[]
document = []
ignore_words = ['?' , '!']
data_file = open('Dummy.json').read()
intents = json.loads(data_file)

In [87]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #print(pattern)
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        document.append((w , intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [88]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

In [89]:
pickle.dump(words , open('words.pkl' , 'wb'))
pickle.dump(classes , open('classes.pkl' , 'wb'))

In [90]:
training = []
output_empty = [0]*len(classes)
for doc in document:
    bag =[]
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in words :
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    
    training.append([bag , output_row])
    

In [91]:
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])

In [92]:
train_x = np.array(train_x)
train_y = np.array(train_y)

In [93]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('chatbot_model_dummy.h5', hist)

print("model created")

Epoch 1/100
669/669 [==============================] - 1s 1ms/step - loss: 3.6796 - accuracy: 0.0538
Epoch 2/100
669/669 [==============================] - 0s 686us/step - loss: 3.4792 - accuracy: 0.0897
Epoch 3/100
669/669 [==============================] - 1s 801us/step - loss: 3.1358 - accuracy: 0.1644
Epoch 4/100
669/669 [==============================] - 0s 742us/step - loss: 2.6898 - accuracy: 0.2930
Epoch 5/100
669/669 [==============================] - 0s 702us/step - loss: 2.3715 - accuracy: 0.3468
Epoch 6/100
669/669 [==============================] - 0s 716us/step - loss: 2.0533 - accuracy: 0.4111
Epoch 7/100
669/669 [==============================] - 0s 694us/step - loss: 1.8394 - accuracy: 0.4813
Epoch 8/100
669/669 [==============================] - 1s 763us/step - loss: 1.6148 - accuracy: 0.5396
Epoch 9/100
669/669 [==============================] - 0s 742us/step - loss: 1.4520 - accuracy: 0.5770
Epoch 10/100
669/669 [==============================] - 0s 718us/step - los

669/669 [==============================] - 1s 765us/step - loss: 0.2377 - accuracy: 0.9238
Epoch 80/100
669/669 [==============================] - 0s 693us/step - loss: 0.3189 - accuracy: 0.9073
Epoch 81/100
669/669 [==============================] - 0s 696us/step - loss: 0.3346 - accuracy: 0.9028
Epoch 82/100
669/669 [==============================] - 0s 692us/step - loss: 0.2209 - accuracy: 0.9387
Epoch 83/100
669/669 [==============================] - 0s 725us/step - loss: 0.2771 - accuracy: 0.9073
Epoch 84/100
669/669 [==============================] - 0s 674us/step - loss: 0.2148 - accuracy: 0.9342
Epoch 85/100
669/669 [==============================] - 0s 723us/step - loss: 0.2979 - accuracy: 0.9088
Epoch 86/100
669/669 [==============================] - 0s 720us/step - loss: 0.2908 - accuracy: 0.9253
Epoch 87/100
669/669 [==============================] - 0s 721us/step - loss: 0.2968 - accuracy: 0.9193
Epoch 88/100
669/669 [==============================] - 0s 713us/step - loss:

In [80]:
classes

['Actions At Home',
 'Actions If At Risk',
 'Children',
 'Citizen',
 'Closed Shops',
 'Contact With Infected',
 'Conversation',
 'Daycare',
 'Doctor',
 'Educating Children On Coronavirus',
 'Fear',
 'Gathering Precautions',
 'Health Authority Recommendations',
 'Illness',
 'Incubation Period',
 'Infected Twice',
 'Infection Prevention',
 'Infection Through Food',
 'Methods of Transmission',
 'Mortality Rate',
 'People At Risk',
 'Precautions',
 'Pregnant',
 'Public Spaces',
 'Public Transport',
 'Purchasing Groceries',
 'Report',
 'Returning From Travel',
 'Sick Child',
 'Strategy',
 'Student',
 'Symptoms',
 'Testing',
 'Treatment',
 'Vacation',
 'Visiting Elderly',
 'What Do We Know About Coronavirus',
 'What Is Coronavirus',
 'goodbye',
 'greeting',
 'thanks']